In [7]:
import pandas as pd
import sqlite3


In [9]:
orders = pd.read_csv('orders.csv')
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [11]:
users = pd.read_json('users.json')
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [15]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

with open('restaurants.sql','r') as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)


In [17]:
print(orders.columns)
print(users.columns)
print(restaurants.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')
Index(['user_id', 'name', 'city', 'membership'], dtype='object')
Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')


In [19]:
merged = pd.merge(
    orders,
    users,
    on='user_id',
    how='left'
)


In [24]:
final_df = pd.merge(
    merged,
    restaurants,
    on='restaurant_id',
    how='left',
    suffixes=('', '_drop')
)


In [26]:
final_df['order_date'] = pd.to_datetime(
    final_df['order_date']
)


C:\Users\Gayathri\AppData\Local\Temp\ipykernel_15820\889747968.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(


In [28]:
final_df['order_date'] = pd.to_datetime(
    final_df['order_date']
)


In [30]:
final_df.isnull().sum()


order_id                0
user_id                 0
restaurant_id           0
order_date              0
total_amount            0
restaurant_name         0
name                    0
city                    0
membership              0
restaurant_name_drop    0
cuisine                 0
rating                  0
dtype: int64

In [118]:
final_df.head()




,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_drop,cuisine,rating,rating_range,quarter
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.0–3.5,1
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1–4.5,1
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3.6–4.0,3
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4.6–5.0,4
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,NaN,4


In [120]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              10000 non-null  int64         
 1   user_id               10000 non-null  int64         
 2   restaurant_id         10000 non-null  int64         
 3   order_date            10000 non-null  datetime64[ns]
 4   total_amount          10000 non-null  float64       
 5   restaurant_name       10000 non-null  object        
 6   name                  10000 non-null  object        
 7   city                  10000 non-null  object        
 8   membership            10000 non-null  object        
 9   restaurant_name_drop  10000 non-null  object        
 10  cuisine               10000 non-null  object        
 11  rating                10000 non-null  float64       
 12  rating_range          9669 non-null   category      
 13  quarter          

In [32]:
final_df.to_csv(
    'final_food_delivery_dataset.csv',
    index=False
)


In [34]:
final_df.groupby('city')['total_amount']\
.sum().sort_values(ascending=False)


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

In [36]:
final_df.groupby('cuisine')['total_amount']\
.sum().sort_values(ascending=False)


cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [38]:
final_df.groupby('membership')['total_amount'].sum()


membership
Gold       3975364.89
Regular    4036259.23
Name: total_amount, dtype: float64

In [40]:
final_df.groupby(
    final_df['order_date'].dt.month
)['total_amount'].sum()


order_date
1     663020.26
2     630867.40
3     716738.98
4     666097.18
5     668428.61
6     610822.93
7     688559.45
8     694987.58
9     653838.07
10    699187.73
11    637772.43
12    681303.50
Name: total_amount, dtype: float64

In [42]:
final_df.groupby('restaurant_name')['rating']\
.mean().sort_values(ascending=False)


restaurant_name
Ruchi Foods South Indian          5.0
Hotel Eatery Pure Veg             5.0
Swagath Delights South Indian     5.0
Grand Tiffins Multicuisine        5.0
Hotel Eatery Family Restaurant    5.0
                                 ... 
Annapurna Restaurant Punjabi      3.0
Royal Restaurant Andhra           3.0
Ruchi Foods Chinese               3.0
Amma Foods Chinese                3.0
Royal Tiffins Pure Veg            3.0
Name: rating, Length: 433, dtype: float64

In [48]:
#📊 Order Trends
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df.groupby(final_df['order_date'].dt.month).size()


order_date
1     831
2     785
3     903
4     812
5     844
6     784
7     859
8     851
9     812
10    863
11    807
12    849
dtype: int64

In [50]:
#User Behavior
final_df.groupby('user_id').size()


user_id
1        1
2       10
3        2
4        4
5        5
        ..
2996     3
2997     5
2998     6
2999     1
3000     2
Length: 2883, dtype: int64

In [52]:
#City Performance
final_df.groupby('city')['total_amount'].sum()




city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [54]:
#Order Trends Over Time
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

final_df.groupby(final_df['order_date'].dt.month)['total_amount'].sum()


order_date
1     663020.26
2     630867.40
3     716738.98
4     666097.18
5     668428.61
6     610822.93
7     688559.45
8     694987.58
9     653838.07
10    699187.73
11    637772.43
12    681303.50
Name: total_amount, dtype: float64

In [56]:
print(final_df.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership', 'restaurant_name_drop',
       'cuisine', 'rating'],
      dtype='object')


In [62]:
#mcq
gold_city = final_df[final_df['membership']=='Gold']\
.groupby('city')['total_amount']\
.sum().sort_values(ascending=False)

print("Q1: City with highest revenue from Gold members:")
print(gold_city.head(1))



Q1: City with highest revenue from Gold members:
city
Chennai    1080909.79
Name: total_amount, dtype: float64


In [64]:
avg_cuisine = final_df.groupby('cuisine')['total_amount']\
.mean().sort_values(ascending=False)

print("\nQ2: Cuisine with highest average order value:")
print(avg_cuisine.head(1))



Q2: Cuisine with highest average order value:
cuisine
Mexican    808.021344
Name: total_amount, dtype: float64


In [66]:
user_spend = final_df.groupby('user_id')['total_amount'].sum()
num_users = len(user_spend[user_spend > 1000])

print("\nQ3: Number of distinct users spending > ₹1000:")
print(num_users)



Q3: Number of distinct users spending > ₹1000:
2544


In [112]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

rating_rev = final_df.groupby('rating_range', observed=False)['total_amount']\
.sum().sort_values(ascending=False)

print("\nQ4: Rating range with highest total revenue:")
rating_rev.head(1)



Q4: Rating range with highest total revenue:


rating_range
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

In [70]:
gold_avg_city = final_df[final_df['membership']=='Gold']\
.groupby('city')['total_amount']\
.mean().sort_values(ascending=False)

print("\nQ5: Gold members — city with highest average order value:")
print(gold_avg_city.head(1))



Q5: Gold members — city with highest average order value:
city
Chennai    808.45908
Name: total_amount, dtype: float64


In [72]:
distinct_rest = final_df.groupby('cuisine')['restaurant_id'].nunique()
revenue_cuisine = final_df.groupby('cuisine')['total_amount'].sum()

print("\nQ6: Cuisine with low number of restaurants but significant revenue:")
print(pd.concat([distinct_rest,revenue_cuisine], axis=1).sort_values(['restaurant_id','total_amount'], ascending=[True,False]))



Q6: Cuisine with low number of restaurants but significant revenue:
         restaurant_id  total_amount
cuisine                             
Chinese            120    1930504.65
Italian            126    2024203.80
Indian             126    1971412.58
Mexican            128    2085503.09


In [74]:
gold_pct = (final_df['membership']=='Gold').mean()*100
gold_pct_rounded = round(gold_pct)

print("\nQ7: Percentage of orders by Gold members:")
print(f"{gold_pct_rounded}%")



Q7: Percentage of orders by Gold members:
50%


In [138]:
options = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

rest_stats = final_df.groupby('restaurant_name')\
.agg(avg_value=('total_amount','mean'),
     total_orders=('order_id','count'))

filtered = rest_stats.loc[options]

answer_q8 = filtered[filtered['total_orders'] < 20]\
.sort_values('avg_value', ascending=False)

print("Q8 Correct Answer from options:")
answer_q8.head(1)


Q8 Correct Answer from options:


,avg_value,total_orders
restaurant_name,,
Ruchi Foods Chinese,686.603158,19


In [132]:
given_restaurants = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

rest_stats = final_df[final_df['restaurant_name'].isin(given_restaurants)]\
.groupby('restaurant_name')\
.agg(avg_value=('total_amount','mean'),
     total_orders=('order_id','count'))

answer_q8 = rest_stats[rest_stats['total_orders'] < 20]\
.sort_values('avg_value', ascending=False)

print("Q8 Answer:")
answer_q8.head(1)


Q8 Answer:


,avg_value,total_orders
restaurant_name,,
Ruchi Foods Chinese,686.603158,19


In [128]:
combo_rev = final_df.groupby(['membership','cuisine'])['total_amount'].sum()

mcq_options = [
    ('Gold','Indian'),
    ('Gold','Italian'),
    ('Regular','Indian'),
    ('Regular','Chinese')
]

filtered_combo = combo_rev.loc[mcq_options]\
.sort_values(ascending=False)

print("Q9 Correct Answer from options:")
filtered_combo.head(1)


Q9 Correct Answer from options:


membership  cuisine
Gold        Italian    1005779.05
Name: total_amount, dtype: float64

In [86]:
final_df['quarter'] = final_df['order_date'].dt.quarter

quarter_rev = final_df.groupby('quarter')['total_amount']\
.sum().sort_values(ascending=False)

print("Q10 Answer: Quarter with highest total revenue:")
quarter_rev.head(1)


Q10 Answer: Quarter with highest total revenue:


quarter
3    2037385.1
Name: total_amount, dtype: float64

In [88]:
#Numerical Answers

In [91]:
print("Total Gold Orders:",
      final_df[final_df['membership']=='Gold'].shape[0])


Total Gold Orders: 4987


In [93]:
hyd_rev = final_df[final_df['city']=='Hyderabad']['total_amount'].sum()
print("Hyderabad Revenue:", round(hyd_rev))


Hyderabad Revenue: 1889367


In [95]:
print("Distinct Users:", final_df['user_id'].nunique())


Distinct Users: 2883


In [97]:
gold_avg = final_df[final_df['membership']=='Gold']['total_amount'].mean()
print("Gold Avg Order Value:", round(gold_avg,2))


Gold Avg Order Value: 797.15


In [99]:
print("Orders rating >=4.5:",
      final_df[final_df['rating']>=4.5].shape[0])


Orders rating >=4.5: 3374


In [101]:
top_city = gold_city.index[0]
count_orders = final_df[(final_df['city']==top_city) &
                        (final_df['membership']=='Gold')].shape[0]

print("Orders in top Gold city:", count_orders)


Orders in top Gold city: 1337
